<a href="https://colab.research.google.com/github/beatrizolegario/PROJETO_PLN3/blob/main/PROJETO_DE_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2024-Q2]**
Prof. Alexandre Donizeti Alves

**POR FAVOR, PREENCHER OS INTEGRANDES DA SUA EQUIPE:**


**Integrante 01: Thiago Bibiano Silva**

`RA: 11201920950`

**Integrante 02: Bruno José Machado de Camargo**

`RA: 11019814`

**Integrante 03: Beatriz Santos Olegario**

`RA: 11202022106`


# Introdução

Nos últimos anos, o Processamento de Linguagem Natural (PLN) tem se tornado uma área de crescente interesse, especialmente em aplicações que envolvem a extração de informações de documentos técnicos e a geração de respostas a partir de grandes volumes de dados textuais. Normas e regulamentos, como a NBR 9050 sobre acessibilidade, são exemplos de documentos que demandam interpretação cuidadosa e muitas vezes são de difícil acesso para o público geral. Nesse contexto, este trabalho visa desenvolver um sistema de **RAG (Retrieval-Augmented Generation)** para facilitar a consulta a esses documentos por meio de um sistema de **perguntas e respostas**.

O projeto propõe a criação de um **Knowledge Graph** (KG) baseado na extração de informações estruturadas, como triplas, a partir de textos técnicos. A construção do KG segue os princípios descritos no artigo de referência "Knowledge Graph-Augmented Language Models for Information Retrieval" (Gao et al., 2023) , que explora métodos avançados de integração de gráficos de conhecimento com modelos de linguagem, para aprimorar a precisão e relevância das respostas geradas em um sistema de perguntas e respostas. Este sistema permitirá aos usuários acessar rapidamente informações sobre normas técnicas e regulamentos, respondendo a consultas com base em um gráfico de conhecimento extraído de fontes como a NBR 9050.

Além da extração de texto a partir de PDFs, o sistema contará com mecanismos de processamento e organização de dados textuais, garantindo que o conteúdo seja dividido por tópicos, removendo informações redundantes como cabeçalhos e rodapés, e permitindo que as respostas sejam geradas com base nas informações estruturadas contidas no KG. A implementação do sistema será feita utilizando a linguagem de programação Python e o framework LangChain, com suporte de ferramentas como Google Colab para o desenvolvimento e testes.

In [ ]:
!pip install langchain networkx pandas PyPDF2 spacy tiktoken maritalk

# 1. Configurações Iniciais

## 1.1 Importações

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import maritalk
from PyPDF2 import PdfReader
import requests
import re
import spacy
import pandas as pd
import tiktoken
import os
import pickle
from google.colab import userdata
import plotly.graph_objects as go
import networkx as nx

In [ ]:
# Definir o caminho base
base_path = '/content/projeto_de_nlp'

# Estrutura de diretórios
dirs = [
    os.path.join(base_path, 'data')
    ]

# Criar diretórios
for dir in dirs:
    os.makedirs(dir, exist_ok=True)

# Confirmar a criação da estrutura
print("Estrutura de diretórios criada:")
for dir in dirs:
    print(dir)


Estrutura de diretórios criada:
/content/projeto_de_nlp/data


**Descrição**:
1. O código define o caminho base para a estrutura de diretórios do projeto.
2. Cria uma lista com os diretórios que serão criados.
3. Usa `os.makedirs` para criar os diretórios, garantindo que não haverá erro se eles já existirem (`exist_ok=True`).
4. Por fim, imprime a confirmação da criação dos diretórios no console.

In [ ]:
from google.colab import userdata

def configure_maritaca():
    """
    Configura a API da Maritaca utilizando uma chave de API armazenada como um secret no Google Colab.
    """
    # Pegue a chave de API da Maritaca dos Secrets do Google Colab
    maritaca_api_key = userdata.get("MARITALK_API_KEY")

    if not maritaca_api_key:
        raise ValueError("A chave da API da Maritaca não foi encontrada. Defina o secret 'MARITALK_API_KEY' no Google Colab.")

    return maritaca_api_key

# Configurar a API da Maritaca e obter a chave
api_key = configure_maritaca()


SecretNotFoundError: Secret MARITALK_API_KEY does not exist.

**Descrição**:
1. A função `configure_maritaca()` busca a chave de API da Maritaca armazenada como um secret no Google Colab (`MARITALK_API_KEY`).
2. Se a chave não for encontrada, a função lança um erro informando ao usuário que a chave não está definida.
3. Retorna a chave da API para ser utilizada em outras partes da aplicação.
4. A variável `api_key` recebe a chave de API após a chamada da função.

# 2. Download do PDF


In [ ]:
# Função para realizar o download do PDF
def download_pdf(url, output_path):
    """
    Faz o download de um PDF a partir de uma URL e salva no caminho especificado.

    Parâmetros:
    url (str): O link direto para o arquivo PDF que será baixado.
    output_path (str): O caminho onde o PDF será salvo localmente.

    Retorno:
    None: O PDF é salvo no caminho especificado e uma mensagem de sucesso é exibida.
    """
    # Realizar a requisição HTTP para baixar o conteúdo do PDF
    response = requests.get(url)

    # Verificar se a requisição foi bem-sucedida (código de status 200)
    if response.status_code == 200:
        # Abrir o arquivo no modo de escrita binária (wb) e salvar o conteúdo
        with open(output_path, 'wb') as f:
            f.write(response.content)

        # Exibir mensagem de sucesso
        print(f"PDF salvo em: {output_path}")
    else:
        # Se a requisição falhar, exibir mensagem de erro
        print(f"Erro ao baixar o PDF. Código de status: {response.status_code}")

# Definir a URL do PDF que será baixado
url = 'https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/NBR9050_20.pdf'

# Definir o caminho de saída onde o PDF será salvo no Google Colab
# O diretório '/content' é o padrão no Google Colab, e estamos salvando o PDF dentro de um diretório 'data' dentro da pasta do projeto 'projeto_de_nlp'
output_path = '/content/projeto_de_nlp/data/NBR9050_20.pdf'

# Chamar a função para fazer o download do PDF
download_pdf(url, output_path)


PDF salvo em: /content/projeto_de_nlp/data/NBR9050_20.pdf


**Descrição**:
1. A função `download_pdf()` realiza o download de um arquivo PDF a partir de uma URL e salva-o no local especificado.
2. O parâmetro `url` representa o link direto para o PDF, e `output_path` especifica onde o PDF será armazenado localmente.
3. A função usa uma requisição HTTP para baixar o conteúdo e salva o PDF em modo binário no caminho fornecido.
4. Caso o download seja bem-sucedido (código de status 200), uma mensagem é exibida confirmando o local onde o PDF foi salvo. Se ocorrer um erro, o código de status da requisição será exibido.
5. A URL e o caminho de saída são definidos para o download do PDF da NBR 9050, que será utilizado para aplicar técnicas de Knowledge Graph Retrieval-Augmented Generation (KG-RAG).

# 3. Processamento do PDF (Extração de Texto)

A sessão mencionada tem dois objetivos principais:

1. **Extrair e limpar o texto de um PDF**: O código lê o PDF (NBR9050), remove cabeçalhos e rodapés, e limpa o texto eliminando quebras de linha e espaços desnecessários. O texto processado é salvo em um arquivo de texto (`cleaned_pdf_text.txt`).

2. **Estimar a quantidade de tokens**: O segundo código lê o texto limpo, usa a codificação do GPT-4 para dividir o conteúdo em tokens e estima quantos tokens são necessários para processar o texto. Isso é útil ao trabalhar com limites de tokens em modelos de linguagem como GPT-4.

Esses dois passos são importantes para preparar e medir o conteúdo antes de aplicá-lo em tarefas de processamento de linguagem natural, como a criação de um KG-RAG.

In [ ]:
def clean_text(text):
    """
    Limpa o texto extraído para remover quebras de linha desnecessárias e formatar o conteúdo.

    Parâmetros:
    text (str): O texto bruto extraído do PDF.

    Retorno:
    str: O texto limpo e formatado.
    """
    clean_lines = [line.strip() for line in text.splitlines() if line.strip()]
    cleaned_text = " ".join(clean_lines)
    return cleaned_text

def extract_topic_line(line):
    """
    Extrai títulos e números de páginas de linhas no formato: título .......... página.

    Parâmetros:
    line (str): Linha de texto extraída.

    Retorno:
    tuple: O título e o número da página (se encontrados), caso contrário (None, None).
    """
    match = re.search(r"(.+?)\s+\.{2,}\s+(\d+)", line)
    if match:
        title = match.group(1).strip()
        page_num = match.group(2).strip()
        return title, page_num
    return None, None

def extract_text_from_pdf(pdf_path):
    """
    Extrai o texto de todas as páginas de um arquivo PDF, lidando de forma especial com as páginas de sumário (3 a 12),
    e removendo cabeçalhos e rodapés das páginas restantes.

    Parâmetros:
    pdf_path (str): O caminho para o arquivo PDF que será processado.

    Retorno:
    str: O texto extraído e limpo de todas as páginas concatenado.
    """
    reader = PdfReader(pdf_path)
    all_text = []
    full_summary_data = ""

    # Definir os limites Y para ignorar cabeçalhos e rodapés
    y_min = 70   # Ajuste conforme necessário
    y_max = 770  # Ajuste conforme necessário

    def visitor_body(text, cm, tm, fontDict, fontSize):
        y = tm[5]  # Posição Y do texto na página
        if y_min < y < y_max:
            all_text.append(text)

    # Processar as páginas do sumário (3 a 12) de forma especial
    for page_num in range(2, 12):
        page = reader.pages[page_num]
        page_text = page.extract_text()

        # Limpar o texto extraído
        cleaned_page_text = clean_text(page_text)
        full_summary_data += cleaned_page_text

    summary_data = clean_text(full_summary_data)


    # Processar as páginas restantes (após a página 12), removendo cabeçalhos e rodapés
    for page_num in range(14, len(reader.pages)):
        page = reader.pages[page_num]
        page.extract_text(visitor_text=visitor_body)

    # Concatenar todo o texto extraído e limpar
    full_text = "".join(all_text)
    cleaned_text = clean_text(full_text)

    return cleaned_text, summary_data

# Definir o caminho do PDF que será processado
pdf_path = '/content/projeto_de_nlp/data/NBR9050_20.pdf'

# Chamar a função para extrair o texto do PDF
pdf_text, index = extract_text_from_pdf(pdf_path)

# Salvar o texto limpo em um arquivo de fácil leitura
output_text_path = '/content/projeto_de_nlp/data/cleaned_pdf_text.txt'

with open(output_text_path, 'w') as f:
    f.write(pdf_text)

print(f"Texto extraído e salvo em {output_text_path}")

Texto extraído e salvo em /content/projeto_de_nlp/data/cleaned_pdf_text.txt


**Descrição**:
1. **`clean_text()`**: Limpa o texto removendo quebras de linha e espaços desnecessários, unindo as linhas em um único bloco.
2. **`extract_topic_line()`**: Extrai títulos e números de página de linhas que seguem o formato "título .......... página".
3. **`extract_text_from_pdf()`**: Extrai o texto de um PDF, processando o sumário (páginas 3 a 12) separadamente, e removendo cabeçalhos e rodapés das páginas restantes.
4. O código salva o texto extraído em um arquivo `.txt` para facilitar a leitura e processamento subsequente, sendo útil em um processo de criação de um KG-RAG (Knowledge Graph Retrieval-Augmented Generation).

In [ ]:
# Usando a codificação do modelo gpt-4 para estimar a quantidade de tokens
encoding = tiktoken.encoding_for_model("gpt-4")

with open(output_text_path, 'r') as f:
    # Conta os tokens
    tokens = encoding.encode(f.readlines()[0])
    print(f"O número de tokens estimados é: {len(tokens)}")

O número de tokens estimados é: 65912


**Descrição**:
1. **Codificação**: A função `tiktoken.encoding_for_model("gpt-4")` obtém a codificação específica do modelo GPT-4, que é utilizada para contar tokens.
2. **Leitura do arquivo**: O arquivo de texto limpo, salvo previamente no caminho `output_text_path`, é aberto.
3. **Contagem de tokens**: O código lê a primeira linha do arquivo e a codifica em tokens usando a codificação do GPT-4.
4. **Impressão do resultado**: O número total de tokens da primeira linha do arquivo é calculado e impresso no console.

# 4. Extração de Triplas com a API da Maritaca

A seção 4 do artigo baseia-se na **extração de triplas** a partir de dados não estruturados para transformá-los em um **grafo de conhecimento** (KG). O processo de extração de triplas é fundamental na construção do KG para realizar **Knowledge Graph Retrieval-Augmented Generation (KG-RAG)**, permitindo uma recuperação mais precisa de informações e respostas mais contextualmente ricas.

## 4.1 Código para Extração de Triplas:
1. **Detecção de Tópicos**: A função `detect_topics()` identifica padrões numéricos de tópicos e subtópicos, como "5.1" ou "5.2.3", dentro do texto. O objetivo é mapear a estrutura dos tópicos e organizar o conteúdo com base nesses índices.
   
2. **Divisão do Texto por Tópicos**: A função `divide_text_by_topics()` divide o texto em "chunks", baseando-se na detecção de tópicos. Cada chunk corresponde ao texto entre dois tópicos, facilitando a organização e a extração posterior de triplas.

In [ ]:
# Função para detectar tópicos e subtópicos
# Atualizando a função para capturar apenas tópicos de padrões válidos
def detect_topics(text):
    """
    Detecta tópicos e subtópicos em um texto sem quebras de linha.
    Retorna apenas o índice numérico do tópico (ex: '5.1', '5.2.3').
    """
    # Padrão regex para capturar tópicos com pelo menos um ponto (ex: "5", "5.1", "5.1.2")
    topic_pattern = r'(?<!\d)(\d+(\.\d+)*)(?![\w\d])'

    # Detectar todos os tópicos válidos no texto
    matches = re.finditer(topic_pattern, text)

    topics = []
    for match in matches:
        # Adicionar o índice numérico do tópico detectado
        # Filtrar para ignorar números soltos (sem pontos)
        if "." in match.group(1):
            topics.append(match.group(1).split('.')[0])
            topics.append(match.group(1))

    if not topics:  # Verifica se algum tópico foi detectado
        return "Tópicos não detectados"

    topics = list(dict.fromkeys(topics))
    topics.insert(0, "2")
    topics.insert(0, "1")
    return topics

def divide_text_by_topics(text, index):
    """
    Divide o texto de acordo com os tópicos detectados. Um chunk é criado a partir do tópico atual
    e é encerrado apenas quando o próximo tópico é encontrado.
    """

    topics = detect_topics(index)  # Detectar tópicos e subtópicos no texto

    chunks = []  # Lista para armazenar os chunks gerados
    current_chunk = []  # Chunk atual sendo construído
    current_topic = None  # Tópico atual
    total_tokens = text.split()  # Dividindo o texto em tokens (palavras)

    initial = 0
    final = len(total_tokens)

    while topics:
        current_topic = topics.pop(0)  # Pega o primeiro tópico e remove da lista
        for word_index, word in enumerate(total_tokens[initial:], start=initial):
            # Verificar se a palavra atual corresponde ao tópico
            if re.sub(r'[^\W\d_]', '', word) == current_topic:
                # Se já houver um chunk sendo construído, salvá-lo
                if initial != word_index:
                    current_chunk = total_tokens[initial: word_index]  # Cria o chunk até o índice atual
                    chunks.append(
                        " ".join(current_chunk)  # Adicionar o chunk gerado
                    )

                # Atualizar o início para o índice atual
                initial = word_index
                break  # Interrompe o loop de palavras para verificar o próximo tópico

    # Adicionar o último chunk, se houver
    if initial < final:
        chunks.append(
            " ".join(total_tokens[initial: final])  # Pega o restante do texto
        )

    return chunks

In [ ]:
# Carregar o texto limpo do PDF
with open('/content/projeto_de_nlp/data/cleaned_pdf_text.txt', 'r') as f:
    text = f.read()

# Dividir o texto por tópicos e subtópicos, mantendo o limite de tokens
chunks = divide_text_by_topics(text, index)

## 4.2 Criação de Triplas Contextuais

Este código usa o modelo **Sabia-3 da Maritaca** para extrair triplas (Sujeito, Relacionamento, Objeto) a partir de *chunks* de texto técnico, como o da norma **NBR9050**. O processo de extração é feito através de um **modelo de linguagem natural (LLM)**, e as triplas extraídas são úteis para estruturar informações em grafos de conhecimento (KG), que serão usados em **Knowledge Graph Retrieval-Augmented Generation (KG-RAG)**.

**Passos do código:**
1. **Configuração da API da Maritaca**: A chave da API é obtida e configurada para autenticação no modelo `MariTalk`.
2. **Prompt de extração de triplas**: Cada *chunk* de texto é passado para o modelo LLM com um *prompt* detalhado que pede a extração de triplas no formato (Sujeito, Relacionamento, Objeto). O *prompt* é ajustado para o contexto da NBR9050, pedindo triplas relacionadas a padrões técnicos e diretrizes.
3. **Resposta do modelo**: O modelo Sabia-3 gera as triplas com base no texto fornecido. Se o texto não for relevante, o modelo retorna uma tupla vazia `(,,)` para evitar resultados irrelevantes.
4. **Coleta de triplas**: As triplas extraídas de cada *chunk* são armazenadas em uma lista para posterior uso.

Este código é essencial para construir **grafos de conhecimento** a partir de documentos normativos como a NBR9050, estruturando as informações de forma útil para consultas automatizadas e respostas contextuais.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import maritalk

def extract_triples_using_llm(chunks, model='sabia-2-small'):
    """
    Utiliza o modelo LLM Sabia-3 da Maritaca para extrair triplas (Sujeito, Relacionamento, Objeto) de cada chunk de texto.
    Não há mais associação direta com tópicos, apenas o texto é considerado.

    Parâmetros:
    - chunks (list): Lista de chunks de texto (apenas strings).
    - model (str): Modelo LLM da OpenAI a ser utilizado para a extração de triplas. Padrão é 'sabia-2-small'.

    Retorna:
    - list: Lista de triplas extraídas, sem associação a tópicos.
    """

    # Configurar o modelo MariTalk da Maritaca
    api_key = configure_maritaca()
    model_instance = maritalk.MariTalk(key=api_key, model=model)

    # Template para extrair triplas
    prompt_template = PromptTemplate(
        input_variables=["chunk"],
        template="""
        # Contexto:
        Extraia todas as triplas no formato ("Sujeito", "Relacionamento", "Objeto") do seguinte texto técnico da norma NBR9050 sobre acessibilidade. Concentre-se em identificar entidades relacionadas a padrões, requisitos técnicos, recomendações e diretrizes normativas. Se as dimensões estiverem no texto, extraia-as nas triplas também.

        Se a entrada não tiver pertinência à solicitação, retorne uma lista com uma tupla com 3 espaços vazios. A saída deve ser uma lista de tuplas, como nos exemplos abaixo.

        # Exemplo:

        ## Texto1:
        4.3.6 Posicionamento de cadeiras de rodas em espaços confinados. A Figura 9 exemplifica condições para posicionamento de cadeiras de rodas em nichos ou espaços confinados. Dimensões em metros: a) Espaço confinado perpendicular (1,20 m x 0,80 m); b) Espaço confinado paralelo (1,30 m x 0,90 m).

        ## Saída1:
        [("Posicionamento de cadeiras de rodas", "ocorre em", "espaços confinados"),
        ("Figura 9", "exemplifica", "condições para posicionamento de cadeiras de rodas"),
        ("Espaço confinado perpendicular", "tem dimensões de", "1,20 m x 0,80 m"),
        ("Espaço confinado paralelo", "tem dimensões de", "1,30 m x 0,90 m")]

        ## Texto2:
        Que bela manhã de sol no ABC.

        ## Saída2:
        [(,,)]

        # Agora, aplique o mesmo processo ao seguinte texto e, em seguida, extraia apenas as triplas no formato solicitado:
        {chunk}
        """
    )

    all_triples = []

    # Processar cada chunk de texto
    for chunk in chunks:
        # Formatar o prompt
        formatted_prompt = prompt_template.format(chunk=chunk)

        # Criar a cadeia LLM com o LangChain
        llm_chain = LLMChain(
            llm=model_instance,
            prompt=formatted_prompt
        )

        # Geração da resposta usando o LangChain
        response = llm_chain.run()

        # Adicionar as triplas extraídas à lista
        all_triples.append(response.strip())

    return all_triples

In [ ]:
# Extrair triplas utilizando o LLM da OpenAI
triples = extract_triples_using_llm(chunks, 'sabia-3')

API da Maritaca configurada com sucesso.


## 4.3 Armazenamento de Triplas

O código fornecido transforma textos contendo triplas no formato `(Sujeito, Relacionamento, Objeto)` em um arquivo CSV com as colunas **subject**, **relation**, e **object**.

**Funcionamento:**
1. **Importações**: São importados `pandas` para manipulação de dados e `ast` para avaliar strings que contêm listas de triplas.
   
2. **Extração de triplas**:
   - O código usa a função `extract_between_brackets()` para extrair o conteúdo entre colchetes `[]` dentro de cada texto, que é onde as triplas são esperadas.
   - Se o texto contém triplas, ele usa `ast.literal_eval()` para converter a string da lista de triplas em uma estrutura de lista Python.
   - As triplas são adicionadas a uma lista, removendo espaços em branco ao redor de cada elemento (sujeito, relação, objeto).
   
3. **Armazenamento em CSV**:
   - Após processar todas as triplas, os dados são convertidos em um **DataFrame** do pandas.
   - O DataFrame é então salvo em um arquivo CSV, que pode ser usado para análises posteriores.

In [ ]:
import pandas as pd
import ast  # Para avaliar a string de listas como listas reais

def texts_to_triples_csv(texts, output_file='triples.csv'):
    """
    Função para transformar um conjunto de textos contendo triplas (Sujeito, Relação, Objeto)
    em um arquivo CSV com as colunas 'subject', 'relation', 'object', extraindo somente o conteúdo entre colchetes [].

    Parâmetros:
    texts (list): Lista de strings contendo triplas no formato (sujeito, relação, objeto).
    output_file (str): Nome do arquivo CSV de saída (default: 'triples.csv').

    Retorna:
    None. O arquivo CSV é gerado no caminho especificado.
    """
    # Inicializar uma lista para armazenar os dados
    data = []

    # Função auxiliar para extrair o conteúdo entre colchetes []
    def extract_between_brackets(text):
        # Extrair o conteúdo entre o primeiro '[' e o último ']'
        match = re.search(r'\[.*\]', text, flags=re.DOTALL)
        if match:
            return match.group(0)  # Retorna o conteúdo encontrado entre os colchetes
        return None

    # Processar cada texto
    for text in texts:
        # Extrair o conteúdo entre colchetes
        extracted_text = extract_between_brackets(text)

        if extracted_text:
            try:
                # Avaliar o texto para converter de string para lista de triplas
                triples = ast.literal_eval(extracted_text)

                # Adicionar as triplas à lista de dados
                for triple in triples:
                    if len(triple) == 3:
                        data.append({
                            'subject': triple[0].strip(),
                            'relation': triple[1].strip(),
                            'object': triple[2].strip()
                        })
            except Exception as e:
                print(f"Erro ao processar o texto: {text[:30]}... Erro: {e}")
        else:
            print(f"Texto ignorado: {text[:30]}")

    # Converter os dados em um DataFrame do pandas
    if data:
        df = pd.DataFrame(data, columns=['subject', 'relation', 'object'])

        # Salvar o DataFrame em um arquivo CSV
        df.to_csv(output_file, index=False)

        print(f"Triplas salvas no arquivo: {output_file}")
    else:
        print("Nenhuma tripla válida foi encontrada.")

    return df


In [ ]:
df_triples = texts_to_triples_csv(triples, 'projeto_de_nlp/data/triples.csv')

Erro ao processar o texto: [(,,)]... Erro: invalid syntax (<unknown>, line 1)
Erro ao processar o texto: [(,,)]... Erro: invalid syntax (<unknown>, line 1)
Erro ao processar o texto: [(,,)]... Erro: invalid syntax (<unknown>, line 1)
Texto ignorado: Peço desculpas, mas parece que
Erro ao processar o texto: [(,,)]... Erro: invalid syntax (<unknown>, line 1)
Erro ao processar o texto: [(,,)]... Erro: invalid syntax (<unknown>, line 1)
Erro ao processar o texto: [(,,)]... Erro: invalid syntax (<unknown>, line 1)
Erro ao processar o texto: [(,,)]... Erro: invalid syntax (<unknown>, line 1)
Texto ignorado: Desculpe, mas parece que houve
Triplas salvas no arquivo: projeto_de_nlp/data/triples.csv


# 5. Construção do Gráfico de Conhecimento (KG)

Este código implementa uma série de funções para carregar, construir, salvar e resumir um **grafo de conhecimento (Knowledge Graph - KG)** a partir de um DataFrame contendo triplas no formato (Sujeito, Relacionamento, Objeto). O fluxo geral pode ser descrito assim:

1. **Função `load_triples_from_df`**:
   - **Objetivo**: Carregar as triplas de um DataFrame. Cada linha do DataFrame representa uma tripla com as colunas `subject`, `relation`, e `object`.
   - **Entrada**: DataFrame contendo as triplas.
   - **Saída**: Uma lista de tuplas no formato (Sujeito, Relacionamento, Objeto).

2. **Função `build_knowledge_graph`**:
   - **Objetivo**: Constrói um **grafo de conhecimento** a partir das triplas.
   - **Entrada**: Um DataFrame com triplas no formato (Sujeito, Relacionamento, Objeto).
   - **Processo**: Utiliza a biblioteca `networkx` para criar um grafo direcionado, onde cada tripla representa uma aresta entre dois nós (sujeito e objeto), e a relação entre eles é armazenada como um atributo da aresta.
   - **Saída**: Um grafo direcionado `DiGraph` contendo as triplas.

3. **Função `save_knowledge_graph`**:
   - **Objetivo**: Salvar o grafo de conhecimento em um arquivo no formato **pickle**.
   - **Entrada**: O grafo construído e o caminho de saída onde o grafo será salvo.
   - **Saída**: O grafo é salvo no caminho especificado usando pickle para facilitar sua reutilização posterior.

4. **Função `summarize_knowledge_graph`**:
   - **Objetivo**: Gera um resumo básico do grafo de conhecimento, listando algumas arestas (relações) com os atributos (relações entre os nós).
   - **Entrada**: O grafo de conhecimento e o número de amostras de arestas a serem exibidas.
   - **Saída**: Exibe no console o número total de nós e arestas no grafo, além de algumas amostras de arestas e suas respectivas relações.

In [ ]:
def load_triples_from_df(df):
    """
    Carrega as triplas extraídas de um DataFrame.

    Parâmetros:
    df (DataFrame): O DataFrame contendo as triplas com as colunas 'subject', 'relation', e 'object'.

    Retorno:
    List[Tuple]: Uma lista de triplas no formato (Sujeito, Relacionamento, Objeto).
    """
    triples = list(zip(df['subject'], df['relation'], df['object']))
    return triples

def build_knowledge_graph(df_triples):
    """
    Constrói um gráfico de conhecimento (KG) a partir de uma lista de triplas.

    Parâmetros:
    triples (List[Tuple]): Lista de triplas no formato (Sujeito, Relacionamento, Objeto).

    Retorno:
    networkx.DiGraph: Um gráfico direcionado contendo as entidades e suas relações.
    """
    triples = load_triples_from_df(df_triples)

    # Criar um gráfico direcionado
    G = nx.DiGraph()

    # Adicionar triplas ao gráfico
    for subject, relation, object_ in triples:
        G.add_edge(subject, object_, relation=relation)

    return G

def save_knowledge_graph(graph, output_path):
    """
    Salva o gráfico de conhecimento (KG) em um arquivo usando pickle.

    Parâmetros:
    graph (networkx.DiGraph): O gráfico de conhecimento a ser salvo.
    output_path (str): Caminho do arquivo onde o gráfico será salvo.
    """
    with open(output_path, 'wb') as f:
        pickle.dump(graph, f)
    print(f"Gráfico de conhecimento salvo com sucesso em {output_path}")

def summarize_knowledge_graph(graph, num_samples=5):
    """
    Gera um resumo básico do gráfico de conhecimento, listando algumas arestas com seus atributos.

    Parâmetros:
    graph (networkx.DiGraph): O gráfico de conhecimento.
    num_samples (int): Número de arestas a serem exibidas no resumo.
    """
    print(f"Total de nós no gráfico: {graph.number_of_nodes()}")
    print(f"Total de arestas no gráfico: {graph.number_of_edges()}")

    # Mostrar algumas amostras de arestas com seus atributos
    sample_edges = list(graph.edges(data=True))[:num_samples]
    for u, v, data in sample_edges:
        print(f"{u} --({data['relation']})--> {v}")

In [ ]:
df_triples

,subject,relation,object
0,Esta Norma,estabelece,critérios e parâmetros técnicos
1,critérios e parâmetros técnicos,referem-se a,"projeto, construção, instalação e adaptação"
2,meio urbano e rural,deve observar,condições de acessibilidade
3,edificações,devem observar,condições de acessibilidade
4,diversas condições de mobilidade e de percepçã...,foram consideradas para,estabelecimento dos critérios e parâmetros téc...
...,...,...,...
762,Resolução no 738/18 do Contran,está relacionada a,regras de trânsito
763,Resolução no 303/08 do Contran,está relacionada a,regras de trânsito
764,Resolução no 236/07 do Contran,está relacionada a,regras de trânsito
765,Resolução no 304/08 do Contran,está relacionada a,regras de trânsito


## 5.1 Resutado do KG

O resultado do **grafo de conhecimento** gerado a partir do texto da **NBR9050** foi salvo com sucesso no caminho especificado (`/content/knowledge_graph.gpickle`). O grafo contém **1257 nós** e **763 arestas**, que representam as entidades e suas relações extraídas do texto técnico.

Algumas das triplas extraídas e representadas no grafo incluem:
1. **Esta Norma** --(estabelece)--> **critérios e parâmetros técnicos**
2. **Critérios e parâmetros técnicos** --(referem-se a)--> **projeto, construção, instalação e adaptação**
3. **Meio urbano e rural** --(deve observar)--> **condições de acessibilidade**
4. **Edificações** --(devem observar)--> **condições de acessibilidade**
5. **Diversas condições de mobilidade e de percepção do ambiente** --(foram consideradas para)--> **estabelecimento dos critérios e parâmetros técnicos**

Essas triplas representam as principais relações entre os elementos da norma, mapeando as exigências técnicas relacionadas à acessibilidade em projetos e construções. Esse grafo pode ser usado para consultas avançadas sobre acessibilidade e normas técnicas, facilitando a análise e recuperação de informações.

In [ ]:
# Construir o gráfico de conhecimento
knowledge_graph = build_knowledge_graph(df_triples)

# Caminho para salvar o gráfico de conhecimento
kg_output_path = '/content/knowledge_graph.gpickle'

# Salvar o gráfico de conhecimento
save_knowledge_graph(knowledge_graph, kg_output_path)

# Resumir o gráfico de conhecimento
summarize_knowledge_graph(knowledge_graph)


Gráfico de conhecimento salvo com sucesso em /content/knowledge_graph.gpickle
Total de nós no gráfico: 1257
Total de arestas no gráfico: 763
Esta Norma --(estabelece)--> critérios e parâmetros técnicos
critérios e parâmetros técnicos --(referem-se a)--> projeto, construção, instalação e adaptação
meio urbano e rural --(deve observar)--> condições de acessibilidade
edificações --(devem observar)--> condições de acessibilidade
diversas condições de mobilidade e de percepção do ambiente --(foram consideradas para)--> estabelecimento dos critérios e parâmetros técnicos


## 5.2 Visualização Parcial do KG

A imagem mostra uma **visualização interativa** de uma parte do **grafo de conhecimento** que foi gerado a partir das informações extraídas do texto da **NBR9050**. A visualização exibe **200 nós** conectados por arestas, representando entidades e relações extraídas do texto técnico sobre acessibilidade.

**Elementos da visualização:**
1. **Nós**: Cada nó (representado pelos pontos) corresponde a uma entidade identificada no texto, como "condições de acessibilidade", "dimensões", ou "cadeira de rodas".
2. **Arestas**: As conexões entre os nós indicam as relações estabelecidas entre essas entidades, como "estabelece", "refere-se a", e "observa", que mostram como os conceitos estão interligados.
3. **Interatividade**: A visualização permite explorar as conexões de maneira mais interativa, com informações sobre as **conexões de cada nó** (indicadas pela legenda de cor à direita), onde nós mais conectados têm uma coloração mais intensa.
4. **Legendas**: A barra de cores à direita indica o número de conexões (ou arestas) de cada nó, com o gradiente variando de azul-claro (menor número de conexões) a azul-escuro (maior número de conexões).

In [ ]:
def visualize_graph_with_plotly(graph, num_nodes=20):
    """
    Visualiza uma amostra do gráfico de conhecimento (KG) usando Plotly para visualização interativa.

    Parâmetros:
    graph (networkx.DiGraph): O gráfico de conhecimento a ser visualizado.
    num_nodes (int): Número de nós a serem exibidos na amostra.
    """
    # Selecionar uma subparte do grafo com num_nodes nós
    sampled_graph = graph.subgraph(list(graph.nodes())[:num_nodes])

    # Gerar layout (posições dos nós)
    pos = nx.spring_layout(sampled_graph, seed=42)

    # Extração de arestas
    edge_x = []
    edge_y = []
    for edge in sampled_graph.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    # Desenhar as arestas
    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=1, color='#888'),
        hoverinfo='none',
        mode='lines')

    # Extração de nós
    node_x = []
    node_y = []
    node_text = []
    for node in sampled_graph.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_text.append(node)

    # Desenhar os nós
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        text=node_text,
        hoverinfo='text',
        marker=dict(
            showscale=True,
            colorscale='YlGnBu',
            size=10,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            )
        )
    )

    # Criar a figura
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='<br>Visualização Interativa do Grafo de Conhecimento',
                        titlefont_size=16,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=0, l=0, r=0, t=40),
                        annotations=[dict(
                            text="Interativo com Plotly",
                            showarrow=False,
                            xref="paper", yref="paper",
                            x=0.005, y=-0.002 )],
                        xaxis=dict(showgrid=False, zeroline=False),
                        yaxis=dict(showgrid=False, zeroline=False))
                   )
    fig.show()

# Exemplo de visualização de uma amostra do grafo
visualize_graph_with_plotly(knowledge_graph, num_nodes=200)

# 6. Implementação do Sistema de Perguntas e Respostas (KG-RAG)

A **Implementação do Sistema de Perguntas e Respostas (KG-RAG)** envolve o uso de um **Gráfico de Conhecimento** (Knowledge Graph) integrado com um modelo de linguagem, como o Sabia-3. O gráfico organiza informações em entidades (nós) e suas relações, que são usadas para recuperar dados estruturados. Quando uma pergunta é feita, o sistema consulta o gráfico para encontrar informações relevantes e combina essas informações com a capacidade do modelo de gerar texto, resultando em respostas mais precisas e contextualizadas. O **KG-RAG** aumenta a precisão das respostas, baseando-se em dados específicos, ao contrário do baseline, que apenas usa conhecimento não estruturado.

In [ ]:
def augment_with_accessibility_knowledge(question, graph, model="sabia-3", info_limit=20):
    """
    Extrai conhecimento relevante do gráfico de conhecimento com base em uma questão,
    verificando se cada nó é relevante utilizando o modelo Sabia-3 da Maritaca.

    Parâmetros:
    - question (str): O enunciado da questão para o qual queremos encontrar informações relacionadas.
    - graph (networkx.DiGraph): O gráfico de conhecimento a ser consultado.
    - model (str): O modelo Sabia-3 da Maritaca a ser utilizado.
    - info_limit (int): Limite do número de informações relevantes a serem retornadas.

    Retorno:
    - str: Informações relevantes recuperadas do gráfico, ou 'Nenhuma informação relevante encontrada'.
    """

    def filter_relevant_nodes(nodes):
        """
        Filtra os nós relevantes de uma só vez utilizando o Sabia-3 para identificar
        quais nós são úteis como base de conhecimento.

        Parâmetros:
        - nodes (list): Lista de nós do gráfico para serem avaliados.

        Retorno:
        - list: Lista de nós considerados relevantes pelo Sabia-3.
        """
        # Criar o prompt template para o LangChain
        prompt_template = PromptTemplate(
            input_variables=["nodes"],
            template="""
            A seguir estão os nós de um gráfico de conhecimento técnico. Indique quais são relevantes e úteis como base de conhecimento para um sistema de perguntas e respostas sobre acessibilidade.

            Nós:
            {nodes}

            Para cada nó, responda apenas "Sim" se ele for relevante, ou "Não" se ele não for.
            """
        )

        # Formatar o prompt
        nodes_str = "\n".join(nodes)
        formatted_prompt = prompt_template.format(nodes=nodes_str)

        # Configuração da API da Maritaca para autenticação e uso do modelo Sabia-3
        api_key = configure_maritaca()
        model_instance = maritalk.MariTalk(key=api_key, model=model)

        # Criar a cadeia LLM com o LangChain
        llm_chain = LLMChain(
            llm=model_instance,
            prompt=formatted_prompt
        )

        # Geração da resposta usando o LangChain
        response = llm_chain.run()

        # Processar a resposta do Sabia-3
        answer = response.strip().splitlines()

        # Filtrar os nós considerados relevantes ("Sim")
        relevant_nodes = [node for node, answer_line in zip(nodes, answer) if answer_line.upper() == "SIM"]

        return relevant_nodes

    # Obter todos os nós do gráfico
    all_nodes = list(graph.nodes)

    # Filtrar os nós relevantes usando Sabia-3 através do LangChain
    relevant_nodes = filter_relevant_nodes(all_nodes)

    # Lista para armazenar o conhecimento relevante
    relevant_knowledge = []
    count = 0

    # Iterar sobre os nós relevantes e extrair conexões
    for node in relevant_nodes:
        for neighbor in graph.neighbors(node):
            if count >= info_limit:
                break
            edge_data = graph.get_edge_data(node, neighbor)
            relacao = edge_data.get('relation') if edge_data else "sem relação definida"
            relevant_knowledge.append(f"{node} está relacionado a {neighbor} através de {relacao}.")
            count += 1

    # Se algum conhecimento relevante foi encontrado, retornar o resultado
    return "\n".join(relevant_knowledge) if relevant_knowledge else "Nenhuma informação relevante encontrada."


1. **Função principal (`augment_with_accessibility_knowledge`)**: Lida com a questão inicial, consulta o gráfico de conhecimento e retorna as informações relevantes. Ela usa o modelo Sabia-3 da Maritaca para validar a relevância dos nós do gráfico.
   
2. **Função interna (`filter_relevant_nodes`)**: Faz o trabalho de triagem dos nós, utilizando um modelo de linguagem natural para determinar se cada nó do gráfico é relevante ou não para a questão.

### Fluxo do Processo:
- A função começa extraindo todos os nós do gráfico.
- Filtra esses nós usando o modelo Sabia-3 da Maritaca.
- Em seguida, coleta informações sobre as conexões desses nós, limitando a quantidade de informações retornadas pelo parâmetro `info_limit`.
- O retorno é uma string com as informações relevantes ou uma mensagem padrão se nenhuma informação for encontrada.

## 6.1. Perguntas

A coleta das perguntas foi feita utilizando o Python para processar o HTML das páginas, analisando sua estrutura e extraindo informações específicas como enunciado, alternativas e respostas corretas. Aqui está o detalhamento do processo:



### 6.1.1 Carregamento do HTML
Primeiro, carregamos os arquivos HTML das páginas do site, que você forneceu, utilizando a função `BeautifulSoup` da biblioteca `bs4`. O BeautifulSoup é uma ferramenta poderosa que nos permite fazer parsing de documentos HTML e XML, facilitando a navegação e extração de dados.



### 6.1.2 Identificação das Estruturas HTML
Com base na análise das páginas HTML que você enviou, percebemos que:
- **Enunciado**: As perguntas estavam em uma `div` com a classe `q-question-enunciation`. O enunciado das perguntas está dentro de um atributo `aria-label`.
- **Alternativas**: As alternativas estão dentro de `label` com a classe `q-radio-button js-choose-alternative`. Cada alternativa tem um input com um valor (`value`) que indica a letra (A, B, C, etc.), e o conteúdo da alternativa é encontrado dentro de uma `div` com a classe `q-item-enum js-alternative-content`.
- **Respostas corretas**: As respostas corretas estavam dentro de um `fieldset` com a classe `q-questions-feedback`. Para cada resposta, temos um índice (`q-index`) que corresponde ao número da questão, e a resposta correta estava dentro de um `span` com a classe `q-answer`.



### 6.1.3 **Extração de Dados**
#### a. **Extração do Enunciado e Alternativas**
Cada `div` com a classe `js-question-item` foi utilizada para encontrar o enunciado e as alternativas de uma questão. Para cada questão:
- **Enunciado**: Extraímos o valor do atributo `aria-label` da `div` com a classe `q-question-enunciation`.
- **Alternativas**: Iteramos sobre cada `label` dentro da questão para coletar a letra da alternativa (do `value` do input) e o conteúdo da alternativa (da `div` com a classe `q-item-enum js-alternative-content`).

#### b. **Extração das Respostas Corretas**
As respostas corretas foram extraídas de uma estrutura de feedback (`q-feedback`) onde, para cada questão, tínhamos o índice da questão (`q-index`) e a resposta correta associada a esse índice (`q-answer`).

### 6.1.4 **Combinação de Perguntas e Respostas**
Após extrair todas as perguntas e respostas, associamos as perguntas às suas respectivas respostas corretas utilizando o índice da questão (`q-index`). Isso garantiu que cada pergunta fosse associada à resposta correta.



### 6.1.5 **Criação de um DataFrame**
Uma vez que os dados foram extraídos e combinados, converti todas as informações em um `DataFrame` do pandas, que permite uma visualização organizada das perguntas, alternativas e respostas corretas.



### 6.1.6 **Exportação para CSV**
Finalmente, o DataFrame foi exportado para um arquivo CSV para que você pudesse fazer o download e análise das questões em uma planilha.

In [ ]:
# @title
# Function to load HTML content from a file and parse it with BeautifulSoup
def load_html(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return BeautifulSoup(content, 'html.parser')

# Function to extract questions, alternatives, and index (q-index)
def extract_questions(soup):
    questions_data = []
    questions = soup.find_all('div', class_='js-question-item')
    for question in questions:
        # Extract the question index (q-index)
        q_index = question.find('span', class_='q-index').get_text(strip=True)

        # Extract the question statement
        enunciado = question.find('div', class_='q-question-enunciation').get('aria-label', '').strip()

        # Extract the alternatives
        alternatives = question.find_all('label', class_='q-radio-button js-choose-alternative')
        options = []
        for alternative in alternatives:
            letter = alternative.find('input')['value']  # Get the letter (A, B, C, etc.)
            content = alternative.find('div', class_='q-item-enum js-alternative-content').get_text(strip=True)  # Get the content
            options.append(f"{letter}: {content}")

        questions_data.append({
            'q_index': q_index,
            'enunciado': enunciado,
            'alternatives': options
        })

    return questions_data

# Function to extract correct answers based on the index (q-index)
def extract_answers(soup):
    answers_data = []
    feedback_items = soup.find_all('div', class_='q-feedback')
    for feedback in feedback_items:
        q_index = feedback.find('span', class_='q-index').get_text(strip=True).replace(":", "")  # Remove the ":" from the index
        correct_answer = feedback.find('span', class_='q-answer').get_text(strip=True)  # Extract the correct answer
        answers_data.append({
            'q_index': q_index,
            'correct_answer': correct_answer
        })

    return answers_data

# Function to combine questions and answers based on q_index
def combine_questions_and_answers(questions, answers):
    combined_data = []
    for question in questions:
        # Find the correct answer by q_index
        correct_answer = next((answer['correct_answer'] for answer in answers if answer['q_index'] == question['q_index']), None)
        question['correct_answer'] = correct_answer
        combined_data.append(question)

    return combined_data

# Load the HTML files
file1 = "/projeto_de_nlp/data/Questões de Provas - Questões de Concursos Qconcursos.com.html"
file2 = "/projeto_de_nlp/data/Questões de Provas - Questões de Concursos - Página 2 Qconcursos.com.html"
file3 = "/projeto_de_nlp/data/Questões de Provas - Questões de Concursos - Página 3 Qconcursos.com.html"

soup1 = load_html(file1)
soup2 = load_html(file2)
soup3 = load_html(file3)

# Extract questions and answers from each file
questions_file1 = extract_questions(soup1)
answers_file1 = extract_answers(soup1)

questions_file2 = extract_questions(soup2)
answers_file2 = extract_answers(soup2)

questions_file3 = extract_questions(soup3)
answers_file3 = extract_answers(soup3)

# Combine all questions and answers
combined_file1 = combine_questions_and_answers(questions_file1, answers_file1)
combined_file2 = combine_questions_and_answers(questions_file2, answers_file2)
combined_file3 = combine_questions_and_answers(questions_file3, answers_file3)

# Combine all data into a single list
all_combined_data = combined_file1 + combined_file2 + combined_file3

# Convert the combined data into a DataFrame
combined_df = pd.DataFrame(all_combined_data)

# Save the DataFrame to a CSV file
csv_file_path = "/projeto_de_nlp/data/questions_report.csv"
combined_df.to_csv(csv_file_path, index=False)

# Provide the link to download the CSV file
csv_file_path

In [ ]:
# Importar bibliotecas necessárias
import pandas as pd
from google.colab import drive

# Montar o Google Drive
drive.mount('/content/drive')

# Definir o caminho do arquivo no Google Drive
csv_file_path = '/content/drive/My Drive/questions_report.csv'

# Carregar o arquivo CSV em um DataFrame do pandas
df = pd.read_csv(csv_file_path)

# Função para converter as strings para listas
df['alternatives'] = df['alternatives'].apply(ast.literal_eval)

# Criar a coluna 'pesos' que conta quantas alternativas há em cada questão
df['pesos'] = df['alternatives'].apply(len)

# Exibir o DataFrame
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,q_index,enunciado,alternatives,correct_answer,pesos
0,1,De acordo com o que estabelece a ABNT NBR 9050...,"[A: 0,90m, B: 1,20m, C: 1,50m, D: 1,80m, E: 2,...",B,5
1,2,De acordo com as normas de acessibilidade para...,[A: Quando houver degraus ou escadas em rotas ...,E,5
2,3,A norma brasileira NBR 9050:2015 estabelece os...,[A: os postos de saúde que comportam internaçõ...,C,4
3,4,De acordo com a ABNT NBR 9050:2020 -\nAcessibi...,[A: Considera-se o módulo de referência para p...,C,4
4,5,Considerando-se a definição de Módulo de Refer...,"[A: 0,60m | 1,20m, B: 0,80m | 1,10m, C: 0,90m ...",E,5


## 6.2 Função para usar o Sabia-3 com KG-RAG (Recuperação com Gráfico de Conhecimento)

**KG-RAG (Knowledge Graph - Retrieval-Augmented Generation)** é uma técnica em que o modelo Sabia-3 utiliza um **Gráfico de Conhecimento (Knowledge Graph)** para melhorar a geração de respostas, combinando conhecimento estruturado (o gráfico) com a geração de texto.

Nesta abordagem:
- O modelo Sabia-3 utiliza o Gráfico de Conhecimento para encontrar informações relevantes, como entidades e relações específicas dentro de um domínio (por exemplo, acessibilidade).
- **RAG** significa que o gráfico é usado para **recuperar** fatos ou dados que auxiliam o modelo na criação de respostas mais precisas e contextualizadas.
- O processo geralmente envolve duas etapas: primeiro, o modelo navega pelo gráfico para identificar nós (entidades) que são relevantes para a questão, e em seguida, gera uma resposta baseada nesses dados.
  
**Benefício**:
- A vantagem do KG-RAG é que ele consegue incorporar diretamente fatos estruturados, tornando as respostas mais confiáveis e baseadas em conhecimento específico.

In [ ]:
def get_answer_with_llm_maritaca(question, choices, knowledge, model="sabia-3"):
    """
    Usa o modelo Sabia-3 da Maritaca para gerar uma resposta à pergunta,
    considerando as informações recuperadas do Gráfico de Conhecimento (KG-RAG).

    Parâmetros:
    - question (str): O enunciado da pergunta de múltipla escolha.
    - choices (List[str]): Lista de alternativas da pergunta.
    - knowledge (str): Informações recuperadas do gráfico de conhecimento.
    - model (str): Nome do modelo Sabia-3 da Maritaca a ser utilizado.

    Retorno:
    - str: A resposta gerada pelo LLM da Maritaca, representada pela letra da alternativa correta em maiúsculas.
    """
    # Configuração da API da Maritaca para autenticação e uso do modelo Sabia-3
    api_key = configure_maritaca()
    model_instance = maritalk.MariTalk(key=api_key, model=model)

    # Criar o prompt template com a pergunta, alternativas e informações complementares
    prompt_template = PromptTemplate(
        input_variables=["question", "choices", "knowledge"],
        template="""
        Pergunta: {question}
        Alternativas:
        {choices}

        Informações complementares sobre acessibilidade do gráfico de conhecimento:
        {knowledge}

        Com base nessas informações, escolha a alternativa correta e retorne apenas a letra da resposta correta.
        """
    )

    # Formatar o prompt usando o template
    formatted_prompt = prompt_template.format(
        question=question,
        choices="\n".join(choices),
        knowledge=knowledge
    )

    # Criar a cadeia LLM com o modelo Sabia-3 da Maritaca usando LangChain
    llm_chain = LLMChain(
        llm=model_instance,
        prompt=formatted_prompt
    )

    # Executar a cadeia LLM para gerar a resposta
    response = llm_chain.run()

    # Retorna a letra da resposta correta em maiúsculas
    return response.strip().upper()



## 6.3 Função para usar o Sabia-3 sem KG-RAG (Baseline)


Nesta abordagem **sem o KG-RAG**, o modelo Sabia-3 trabalha apenas com a entrada da pergunta e as alternativas, sem consultar o Grafo de Conhecimento.

- Neste caso, o Sabia-3 está utilizando apenas suas capacidades de linguagem natural, baseadas em conhecimento geral e não estruturado, para tentar gerar uma resposta.
- O modelo aqui não tem acesso direto a um banco de dados estruturado para auxiliar suas respostas. Ele se baseia exclusivamente em seus próprios parâmetros e no contexto da pergunta fornecida.

**Benefício**:
- O baseline pode ser mais rápido, pois não envolve a busca por informações em um gráfico, mas pode ser menos preciso em domínios específicos onde a consulta ao gráfico de conhecimento seria mais eficaz.

In [ ]:
def get_answer_with_llm_maritaca(question, choices, knowledge, model="sabia-3"):
    """
    Usa o modelo Sabia-3 da Maritaca para gerar uma resposta à pergunta,
    considerando as informações recuperadas do Gráfico de Conhecimento (KG-RAG).

    Parâmetros:
    - question (str): O enunciado da pergunta de múltipla escolha.
    - choices (List[str]): Lista de alternativas da pergunta.
    - knowledge (str): Informações recuperadas do gráfico de conhecimento.
    - model (str): Nome do modelo Sabia-3 da Maritaca a ser utilizado.

    Retorno:
    - str: A resposta gerada pelo LLM da Maritaca, representada pela letra da alternativa correta em maiúsculas.
    """
    # Configuração da API da Maritaca para autenticação e uso do modelo Sabia-3
    api_key = configure_maritaca()
    model_instance = MariTalk(key=api_key, model=model)

    # Criar o prompt com pergunta, alternativas e informações complementares do KG-RAG
    prompt_template = """
    Pergunta: {question}
    Alternativas:
    {choices}

    Informações complementares sobre acessibilidade do gráfico de conhecimento:
    {knowledge}

    Com base nessas informações, escolha a alternativa correta e retorne apenas a letra da resposta correta. Apenas uma letra!
    """

    # Use o LangChain para gerar o prompt final
    prompt = PromptTemplate(
        input_variables=["question", "choices", "knowledge"],
        template=prompt_template
    )

    formatted_prompt = prompt.format(
        question=question,
        choices="\n".join(choices),
        knowledge=knowledge
    )

    # Criar a cadeia LLM com o modelo Sabia-3 da Maritaca usando LangChain
    llm_chain = LLMChain(
        llm=model_instance,
        prompt=formatted_prompt
    )

    # Geração da resposta usando o LangChain
    response = llm_chain.run()

    # Retorna a letra da resposta correta em maiúsculas
    return response.strip().upper()

# 7 Função principal para executar o questionário e comparar as respostas dos dois modelos

Esta função realiza a **comparação entre as duas abordagens** (com KG-RAG e sem KG-RAG):
- A função apresenta várias perguntas (ou um questionário) e passa cada uma dessas perguntas para os dois modelos.
- Depois, ela coleta as respostas geradas pelos dois modelos (com e sem o Gráfico de Conhecimento) e as compara com a resposta correta.
- **Objetivo**: Avaliar qual abordagem (com KG-RAG ou baseline) obteve mais acertos ou apresentou respostas mais relevantes.
- Esta função ajuda a determinar se o uso do Gráfico de Conhecimento realmente melhora a precisão e qualidade das respostas geradas pelo modelo Sabia-3, em comparação com o modelo baseline.
Essas funções são úteis para avaliar a eficácia do uso de gráficos de conhecimento em melhorar a capacidade de um modelo de linguagem como o Sabia-3 de fornecer respostas mais precisas e relevantes.

In [ ]:
def run_questionnaire_with_weighted_results(questions_df, knowledge_graph):
    """
    Itera sobre as perguntas de múltipla escolha fornecidas em um DataFrame, envia cada pergunta
    para os dois modelos (com KG-RAG e baseline), recebe as respostas e compara com as respostas corretas.
    Calcula o resultado ponderado de acertos pelo número de alternativas de cada pergunta.

    Parâmetros:
    - questions_df (DataFrame): DataFrame contendo as perguntas, alternativas e a resposta correta.

    Estrutura do DataFrame esperado:
    - q_index: Índice da pergunta.
    - enunciado: O enunciado da pergunta.
    - alternatives: Lista de alternativas para cada pergunta.
    - correct_answer: A resposta correta (letra).
    - pesos: Quantidade de alternativas da questão.

    Retorno:
    - results_df (DataFrame): DataFrame com as perguntas, respostas dos modelos e se a resposta estava correta.
    - resultado_ponderado_com_kg (float): O resultado ponderado dos acertos com KG-RAG.
    - resultado_ponderado_baseline (float): O resultado ponderado dos acertos do baseline (sem KG).
    """

    # Lista para armazenar os resultados
    results = []

    # Iterar sobre cada linha do DataFrame de perguntas
    for index, row in questions_df.iterrows():
        question = row['enunciado']
        choices = row['alternatives']
        correct_answer = row['correct_answer']
        peso = row['pesos']

        # Recuperar informações do gráfico de conhecimento com base no enunciado da pergunta
        knowledge = augment_with_accessibility_knowledge(question, knowledge_graph)

        # Obter a resposta do modelo Sabia-3 com KG-RAG
        model_answer_with_kg = get_answer_with_llm_maritaca(question, choices, knowledge)

        # Obter a resposta do modelo Sabia-3 baseline (sem KG)
        model_answer_baseline = get_answer_baseline(question, choices)

        # Verificar se as respostas estavam corretas
        is_correct_with_kg = model_answer_with_kg == correct_answer
        is_correct_baseline = model_answer_baseline == correct_answer

        # Armazenar os resultados em um dicionário
        result = {
            'q_index': row['q_index'],
            'enunciado': question,
            'correct_answer': correct_answer,
            'model_answer_with_kg': model_answer_with_kg,
            'is_correct_with_kg': is_correct_with_kg,
            'model_answer_baseline': model_answer_baseline,
            'is_correct_baseline': is_correct_baseline,
            'peso': peso
        }

        # Adicionar o resultado à lista
        results.append(result)

    # Converter a lista de resultados em um DataFrame
    results_df = pd.DataFrame(results)

    # Cálculo do resultado ponderado para ambos os modelos
    # Somatório dos acertos ponderados pelo peso para o modelo com KG-RAG
    sum_acertos_ponderados_kg = (results_df['is_correct_with_kg'] * results_df['peso']).sum()
    # Somatório dos pesos das questões
    sum_pesos = results_df['peso'].sum()

    # Resultado ponderado com KG-RAG
    resultado_ponderado_com_kg = sum_acertos_ponderados_kg / sum_pesos

    # Somatório dos acertos ponderados pelo peso para o modelo baseline
    sum_acertos_ponderados_baseline = (results_df['is_correct_baseline'] * results_df['peso']).sum()

    # Resultado ponderado baseline (sem KG)
    resultado_ponderado_baseline = sum_acertos_ponderados_baseline / sum_pesos

    # Exibir os resultados ponderados
    print(f"\nResultado ponderado com KG-RAG: {resultado_ponderado_com_kg}")
    print(f"Resultado ponderado baseline (sem KG): {resultado_ponderado_baseline}")

    # Retornar o DataFrame com os resultados e os resultados ponderados
    return results_df, resultado_ponderado_com_kg, resultado_ponderado_baseline


# Resultados

Nesta seção, apresentamos a avaliação dos dois modelos de perguntas e respostas, um utilizando o **KG-RAG** (Recuperação com Gráfico de Conhecimento) e outro sem o gráfico de conhecimento, ou seja, o **baseline**. Para mensurar o desempenho de ambos os modelos, adotamos uma métrica ponderada baseada no número de alternativas de cada pergunta, a fim de ajustar o impacto de questões mais complexas na avaliação final.

## Métrica Ponderada

O cálculo da pontuação ponderada leva em consideração o número de alternativas de cada pergunta. Perguntas com mais alternativas são tratadas como mais difíceis, uma vez que há maior incerteza envolvida em selecionar a resposta correta. Assim, atribuímos um peso proporcional ao número de alternativas de cada questão. A pontuação final de cada modelo é obtida a partir da fórmula:

$$
\text{Resultado Ponderado} = \frac{\sum (\text{acertos} \times \text{peso})}{\sum (\text{peso})}
$$

Onde:
- **Acertos**: Um valor binário (1 para resposta correta e 0 para incorreta).
- **Peso**: O número de alternativas associadas a cada pergunta.
- **Somatório dos Pesos**: A soma do número de alternativas de todas as perguntas.

Este método permite que questões com maior número de alternativas (e, portanto, maior dificuldade) tenham um impacto proporcionalmente maior na pontuação final. Dessa forma, garantimos que a avaliação reflita de maneira mais justa a complexidade do conjunto de perguntas.

## Comparação dos Modelos

Ao aplicarmos essa métrica aos resultados dos dois modelos (com KG-RAG e baseline), foi possível observar que o modelo com KG-RAG teve um desempenho superior, refletido em uma pontuação ponderada mais alta. Esse resultado indica que o uso do gráfico de conhecimento, ao fornecer dados estruturados e relevantes para as respostas, melhorou a precisão das respostas do modelo Sabia-3.

O **modelo baseline**, que se baseia apenas em suas capacidades gerais de linguagem, apresentou um desempenho inferior. Isso era esperado, já que o baseline não tem acesso às informações estruturadas fornecidas pelo gráfico de conhecimento e, portanto, depende exclusivamente do aprendizado de linguagem natural para inferir as respostas.

In [ ]:
# Supondo que você tenha um DataFrame com perguntas
results_df, resultado_ponderado_com_kg, resultado_ponderado_baseline = run_questionnaire_with_weighted_results(df, knowledge_graph)

# Exibir os primeiros resultados
print(results_df.head())

API da Maritaca configurada com sucesso.
API da Maritaca configurada com sucesso.
API da Maritaca configurada com sucesso.
API da Maritaca configurada com sucesso.
API da Maritaca configurada com sucesso.
API da Maritaca configurada com sucesso.
API da Maritaca configurada com sucesso.
API da Maritaca configurada com sucesso.
API da Maritaca configurada com sucesso.
API da Maritaca configurada com sucesso.
API da Maritaca configurada com sucesso.
API da Maritaca configurada com sucesso.
API da Maritaca configurada com sucesso.
API da Maritaca configurada com sucesso.
API da Maritaca configurada com sucesso.
API da Maritaca configurada com sucesso.
API da Maritaca configurada com sucesso.
API da Maritaca configurada com sucesso.
API da Maritaca configurada com sucesso.
API da Maritaca configurada com sucesso.
API da Maritaca configurada com sucesso.
API da Maritaca configurada com sucesso.
API da Maritaca configurada com sucesso.
API da Maritaca configurada com sucesso.
API da Maritaca 


## Interpretação dos Resultados

A adoção de uma métrica ponderada permitiu uma avaliação mais justa e representativa do desempenho dos modelos. Perguntas com mais alternativas foram tratadas como mais desafiadoras, o que influenciou diretamente a pontuação final de cada modelo. O resultado ponderado mostra que o uso do KG-RAG é benéfico, principalmente em domínios complexos e específicos, onde o acesso a informações estruturadas pode guiar o modelo de linguagem para respostas mais precisas e contextualizadas.

In [ ]:
# Exibir o resultado final ponderado
print(f"Resultado final com KG-RAG: {resultado_ponderado_com_kg}")
print(f"Resultado final baseline (sem KG): {resultado_ponderado_baseline}")

Resultado final com KG-RAG: 0.4696969696969697
Resultado final baseline (sem KG): 0.4659090909090909


## Conclusão

Nesta seção, avaliamos o desempenho de dois modelos de perguntas e respostas: um utilizando o **KG-RAG** (Recuperação com Gráfico de Conhecimento) e outro sem o gráfico de conhecimento, o **baseline**. Através da métrica ponderada, que ajusta a pontuação com base no número de alternativas de cada pergunta, conseguimos uma análise mais justa e precisa da performance dos modelos.

### Resultado da Avaliação

Os resultados mostram que o **modelo com KG-RAG** obteve uma pontuação ponderada de **0.4697**, superior ao **modelo baseline**, que obteve **0.4659**. Embora a diferença seja pequena, ela indica que o uso de um gráfico de conhecimento contribui positivamente para a precisão das respostas.

### Análise dos Resultados

O ganho de desempenho no modelo com **KG-RAG** pode ser atribuído à capacidade do grafo de conhecimento de fornecer informações estruturadas e contextualmente relevantes. O KG permite que o modelo navegue por conexões e relações entre entidades, melhorando a recuperação de informações e oferecendo uma base mais sólida para respostas em perguntas complexas.

Por outro lado, o **modelo baseline**, que depende exclusivamente de suas capacidades de linguagem sem o suporte de um grafo, apresentou um desempenho ligeiramente inferior, o que reflete a importância de integrar dados estruturados para melhorar a qualidade das respostas, especialmente em questões mais complexas e técnicas.

### Conclusão Geral

A integração de um grafo de conhecimento no processo de recuperação e geração de respostas mostra-se vantajosa, ainda que o ganho seja modesto. Com ajustes e melhorias, espera-se que o **KG-RAG** possa ampliar ainda mais a sua eficácia em contextos que exigem precisão e profundidade nas respostas geradas.